In [9]:
from aocd import get_data, submit
import numpy as np
import sys
import itertools
from collections import defaultdict, Counter
import re

day = 20
year = 2022

data = get_data(day=day, year=year)

# data = '\n'.join(re.split(r'[\.|:|\n]', data))
# data = ">>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>"
# print(data)

In [6]:
data[:10]

'-624\n8483\n'

Part 1: 1599
Part 2: 14112


In [48]:
print(sum((a+1)*b for a,b in vals))
# vals

# 1605 too high

126


In [74]:
vals[0][1] * vals[1][1] * vals[2][1]

# 12096 too low

# 14112 -> correct!

14976

In [68]:
49*18*16

12096

In [ ]:
# 1000000000000 
(1000000000000-37) // 35, (1000000000000-37) % 35

In [ ]:
Cycle starts: 37 bricks | height: 63
full cycle: 72 tot bricks | tot height: 116
cycle brick length: 35


In [ ]:
# min_flows = dict()
# # nx.bfs_tree(G, 'AA', depth_limit=30)
#     # print(t)
# cur_states = {('AA', (), 0)}
# tot_minutes = 30

# for minute in range(1, tot_minutes+1):
#     next_states = set()
#     for cur_node, open_valves, flow in cur_states:
#         flow_rate = sum(flows[n] for n in open_valves)
#         min_flow = flow + flow_rate * (tot_minutes-minute)
#         # print(f"Exploring: {cur_node} {open_valves} {flow}")
#         if min_flows.get(cur_node, -1) >= min_flow:
#             # print(f" -> Pruning: {flow} + {tot_minutes-minute} * {flow_rate} -> {min_flow} <= {min_flows[cur_node]}")
#             continue
#         # print(f" Adding min_flow: {cur_node} -> {min_flow}")
#         min_flows[cur_node] = min_flow
#         flow += flow_rate
#         if (cur_node not in open_valves) and flows[cur_node]:
#             next_states.add((cur_node, tuple(set([*open_valves, cur_node])), flow))
#         for next_node in G.neighbors(cur_node):
#             next_states.add((next_node, open_valves, flow))
#     if len(next_states) == 0:
#         pprint(cur_states)
#         pprint(min_flows)
#     print(f"Minute {minute}: max flow = {max(f for _,_,f in next_states)} ({len(next_states)} active states)")
#     cur_states = next_states
    
#     # pprint(min_flows)

In [ ]:
# sum(v for k,v in flows.items() if k in ('BB', 'DD', 'HH'))

    # ('BB', 'HH') ('DD', 'JJ
# flows

In [ ]:
all_valves = {k for k,v in flows.items() if v}
min_flows = dict()
# nx.bfs_tree(G, 'AA', depth_limit=30)
    # print(t)
cur_states = {(('AA', 'AA'), (), 0, ())}
tot_minutes = 26

for minute in range(1, tot_minutes+1):
    print(f"Minute {minute}: max flow = {max([0, *min_flows.values()])} ({len(cur_states)} active states)\n")
        
    next_states = set()
    for cur_nodes, open_valves, flow, prev_nodes in  sorted(cur_states, key=lambda x: x[2], reverse=True):
        flow_rate = sum(flows[n] for n in open_valves)
        flow += flow_rate
        
        min_flow = flow + flow_rate * (tot_minutes-minute)

        if min_flows.get(cur_nodes, -1) >= min_flow:
            continue
        
        min_flows[cur_nodes] = min_flow
        
        if len(open_valves) == len(all_valves):
            continue
        
            
        next_states_n = [[], []]
        for i,cur_node in enumerate(cur_nodes):
            next_states_n[i] = [([(next_node, []) 
                                 for next_node in G.neighbors(cur_node)
                                 if next_node not in (prev_nodes)]
                                + ([(cur_node, [cur_node])] if flows[cur_node] and cur_node not in open_valves else []))
                                
            ]
        debug = {(
            tuple(sorted([cur_node1, cur_node2])), 
            tuple(sorted(set([*valve1, *valve2, *open_valves]))), 
            flow,
            (cur_nodes)
        )
                            for cur_node1, valve1 in next_states_n[0]
                            for cur_node2, valve2 in next_states_n[1]
                            }

        next_states.update(debug)
        
    cur_states = next_states

    if len(cur_states) == 0:
        print(f"No more moves.")
        break
    
print(f" Max flow: {max(min_flows.values())}")
    # pprint(min_flows)
    # print(max(min_flows.values()), '\n')
    # 2532
    # 2349
    # 2715
    
    # ('BB', 'HH') ('DD', 'JJ

In [159]:
from aocd import get_data, submit
import numpy as np
import sys
import itertools
from collections import defaultdict, Counter
import re

day = 18
year = 2022

data = get_data(day=day, year=year)


In [168]:
cubes = {tuple([int(i) for i in l.split(',')]) for l in data.split()}

faces = len(cubes) * 6
faces -= sum( tuple(cc + d*pp for cc,pp in zip(c,p)) in cubes
             for c in cubes for d in (-1, 1) for p in ((1,0,0),(0,1,0),(0,0,1)))

print("Part 1:", faces)


4364

In [175]:
m = max(max(cubes, key=lambda x: x[i])[i] for i in range(3))

steam = set(
    [(a, b, c) for a in (-1,m+2) for b in range(-1, m+2) for c in range(-1, m+2)]
+   [(b, c, a) for a in (-1,m+2) for b in range(-1, m+2) for c in range(-1, m+2)]
+   [(c, a, b) for a in (-1,m+2) for b in range(-1, m+2) for c in range(-1, m+2)]
)

expanded = True
while expanded:
    expanded = False
    for x in range(0, m+1):
        for y in range(0, m+1):
            for z in range(0, m+1):
                if (x,y,z) in cubes or (x,y, z) in steam:
                    continue
                for d in (-1,1):
                    if (x+d,y,z) in steam or (x,y+d,z) in steam or (x,y,z+d) in steam:
                        steam.add((x,y,z))
                        expanded = True
                        break

cooling = {(x,y,z)
    for x in range(0, m+1)
        for y in range(0, m+1)
            for z in range(0, m+1)
                if (x,y,z) not in steam }

faces = len(cubes) * 6
faces -= sum( tuple(cc + d*pp for cc,pp in zip(c,p)) in cooling
             for c in cubes for d in (-1, 1) for p in ((1,0,0),(0,1,0),(0,0,1)))


print("Part 2:", faces)

Part 2: 2508
